# Notebook for USASpending files

### This notebook will provide the steps for a pipeline script in Python to handle data

In [ ]:
# API pull for bulk download file URL

import requests

params =  {"agency": 22, "fiscal_year": 2016, "type": "contracts"}
r = requests.post("https://api.usaspending.gov/api/v2/bulk_download/list_monthly_files//", data=params)

with open('Treasury.txt', 'wb') as file:
    file.write(r.content)

In [32]:
# Import necessary libraries
import numpy as np
import pandas as pd
import re
import datetime

# Create small data set to pull date-parsing column names
col_list = list(pd.read_csv(
    '_Data/TREAS/api_bulk_listmonthlyfiles/2015_020_Contracts_Full_20181212_1.csv').columns)

# Extract date column names with regular expression
date_list = [col for col in col_list if re.search(r'date\b', col) is not None]

# Build full dataframe with parsed dates
df = pd.read_csv('_Data/TREAS/api_bulk_listmonthlyfiles/2015_020_Contracts_Full_20181212_1.csv',
                 parse_dates=date_list, infer_datetime_format=True)

# Parse the dates of problematic column: period_of_performance_start_date
df.loc[:, 'period_of_performance_start_date'] = pd.to_datetime(
    df['period_of_performance_start_date'], errors='coerce')

# Forward fill NaN values in parent_award_id with award_id_piid values
df.loc[:, 'parent_award_id'] = df['parent_award_id'].fillna(df['award_id_piid'])

# Create dataframes of vehicle and work
vehicle = pd.read_csv('_Data/TREAS/ContractVehicles.csv', 
                      names=['parent_award_id', 'contract_vehicle'])
work = pd.read_csv('_Data/TREAS/Work.csv')
work.columns = ['parent_award_id', 'award_id_piid', 'work']

# Join vehicle and work to file
df_new = df.merge(vehicle, on='parent_award_id')
df_new = pd.merge(df, work, on=['parent_award_id', 'award_id_piid'], how='left')

# Create fiscal year by adding '1' to months after September
# Need to verify WHICH column is used for fiscal year
def to_fiscal(date_column, index=df_new.index):
    df_new['fiscal_year'] = pd.to_datetime(date_column)
    fiscal_year = []
    for date in date_column:
        if date.month > 9:
            fiscal_year.append(date.year + 1)
        else:
            fiscal_year.append(date.year)
    return pd.Series(fiscal_year, index=index)
df_new['fiscal_year'] = to_fiscal(df_new['action_date'])

# Extract columns names with 't' and 'f' values only
bool_list = []
for col in df_new:
    if col != 'fiscal_year':
        if len(df_new[col].unique()) == 2 and df_new[col].dtype != '<M8[ns]':
            if 't' and 'f' in df[col].unique():
                bool_list.append(str(col))
        if len(df_new[col].unique()) < 2 and df_new[col].dtype != '<M8[ns]':
            if 't' in df[col].unique():
                df_new.loc[:, col] = True
            if 'f' in df[col].unique():
                df_new.loc[:, col] = False
            
# Overwrite columns with only 't' and 'f' with boolean values
for name in bool_list:
    df_new.loc[:, name] = pd.get_dummies(df_new[name]).astype('bool')['t']
    
# Return Excel xlsm file
# df_new.to_excel('Output.xlsm', index=False)

C:\Users\seapea\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
C:\Users\seapea\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [33]:
df_new.head(10)

award_id_piid modification_number  transaction_number  \
0     TFSAHUD15K0007                   0                   0   
1     TFSACFP14K0001                   2                   0   
2       TMDN15MX0593                   0                   0   
3  TOSOFR14D00060001                   1                   0   
4     TFSAADF14K0010                 700                   0   
5      TIRNO15T00068                   0                   0   
6      TFMSHQ13K0023                   7                   0   
7      TFMSHQ12K0021                  14                   0   
8  TOSOFR14D00060002                   2                   0   
9      TIRNO13T00038                   5                   0   

  parent_award_agency_id     parent_award_agency_name parent_award_id  \
0                   4732  FEDERAL ACQUISITION SERVICE      GS06F1197Z   
1                   4732  FEDERAL ACQUISITION SERVICE      GS06F0806Z   
2                    NaN                          NaN    TMDN15MX0593   
3                   4732  FEDERAL ACQUISITION SERVICE      GS06F0723Z   
4                   4732  FEDERAL ACQUISITION SERVICE      GS06F0753Z   
5                   4732  FEDERAL ACQUISITION SERVICE      GS06F0773Z   
6                   4732  FEDERAL ACQUISITION SERVICE      GS06F0955Z   
7                   4730  FEDERAL ACQUISITION SERVICE      GS23F0230P   
8                   4732  FEDERAL ACQUISITION SERVICE      GS06F0723Z   
9                   4732  FEDERAL ACQUISITION SERVICE      GS06F0884Z   

  parent_award_modification_number  federal_action_obligation  \
0                                2                   65943.08   
1                                0                       0.00   
2                              NaN                    5670.00   
3                                0                   73773.76   
4                                0                       0.00   
5                                0                   79798.47   
6                                3                 -100445.89   
7                                0                 1402988.99   
8                                0                  630000.00   
9                                0                  299961.47   

   total_dollars_obligated  base_and_exercised_options_value     ...      \
0                 65943.08                          65943.08     ...       
1               3704818.88                              0.00     ...       
2                      NaN                           5670.00     ...       
3                442851.66                          73773.76     ...       
4                510112.10                              0.00     ...       
5                402998.43                         423093.75     ...       
6               3942566.11                        -100445.89     ...       
7                      NaN                        1402988.99     ...       
8                906595.27                         630000.00     ...       
9               2344153.18                        1325447.11     ...       

   veterinary_college  dot_certified_disadvantage  \
0               False                       False   
1               False                       False   
2               False                       False   
3               False                       False   
4               False                        True   
5               False                       False   
6               False                        True   
7               False                       False   
8               False                       False   
9               False                        True   

   self_certified_small_disadvantaged_business small_disadvantaged_business  \
0                                        False                         True   
1                                        False                         True   
2                                         True                        False   
3                                        Fals

In [ ]:
'''
Remove rows with product code nn99
'''

In [44]:
df_again = pd.merge(df_new, df_new.groupby('work')[['period_of_performance_potential_end_date']].max(),
         on='work', how='left')

In [50]:
df_again['period_of_performance_potential_end_date_y'].value_counts()

2105-12-31    783
2017-09-30     55
2018-09-27     46
2018-09-30     25
2019-05-31     23
2018-09-23     19
2017-07-19     15
2018-05-25     14
2018-03-31     13
2017-10-22     13
2017-09-28     13
2019-04-30     12
2020-01-13     12
2019-08-31     12
2019-06-27     11
2022-06-28     11
2022-12-02      7
2022-04-27      7
2016-02-15      7
2017-08-28      6
2017-03-28      6
2018-03-10      6
2017-12-01      6
2023-12-31      5
2019-12-11      5
2020-04-14      5
2019-09-30      5
2017-07-31      4
2017-05-17      4
2016-04-16      4
2017-04-21      3
2016-01-31      3
2020-09-14      3
2020-12-31      3
2016-11-30      3
2017-11-17      3
2016-08-31      3
2019-12-17      2
2020-09-30      2
2016-04-29      2
2021-03-27      2
2018-03-28      2
2018-02-18      2
2021-03-01      2
2020-09-29      2
2017-03-31      2
2019-03-31      1
2018-09-20      1
2020-09-22      1
2019-10-31      1
2011-09-29      1
2019-09-03      1
2022-10-31      1
2018-12-31      1
2021-03-13      1
2018-03-26